In [2]:
from pyspark.sql.types import *
from pyspark.sql import functions as f
import requests
import pandas as pd 
import datetime

StatementMeta(spark001, 0, 3, Finished, Available)

In [3]:
# import raddit library
try:
    import praw
except:
    get_ipython().system('pip install praw')
    import praw
    
from praw.models import MoreComments

StatementMeta(spark001, 0, 4, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 27.9 MB/s eta 0:00:00


In [6]:
# set up The Reddit Instance
def set_up_reddit():
    user_agent='scraper 1.0 by /u/Adrian_10511'
    reddit= praw.Reddit(
        client_id='ZSsuFt18m9u6PnXJJwjF1A',
        client_secret='cKPHnxi_E9gr7HM5DKdO-XU7nFdO2w',
        user_agent = user_agent
    )
    return reddit
    
# set up query
def search_post(query,reddit,subreddit='stocks',limit_=100):
    search_query=query
    subreddit_name = subreddit
    results = reddit.subreddit(subreddit_name).search(search_query, limit=limit_)
    
    # change result into dataframe
    data=[]
    for submission in results:
        data.append({
            'Title': submission.title,
            'Score': submission.score,
            'ID': submission.id,
            'URL': submission.url,
            'Created': submission.created_utc,
            'Number of Comments': submission.num_comments
        })
    post_df = pd.DataFrame(data)
    post_df['Created'] = pd.to_datetime(post_df['Created'], unit='s')
    return post_df

def extract_comment(df,symbol,day=30):
    end_date = datetime.datetime.now()
    start_date = end_date-datetime.timedelta(days=day)
    # Filter the post based on the date range
    filtered_df = df[(df['Created'] >= start_date) & (df['Created'] <= end_date)].sort_values(by=['Created'],ascending=False)

    # extract df 
    cmt_list=[]
    for post_id in filtered_df['ID']:
        for top_level_comment in reddit.submission(post_id).comments:
            if isinstance(top_level_comment, MoreComments):
                continue
            cmt_list.append({
                'created_utc':top_level_comment.created_utc,
                'comment':top_level_comment.body
                })
            
    # turn into dataframe 
    cmt_df = pd.DataFrame(cmt_list)
    cmt_df['created_utc'] = pd.to_datetime(cmt_df['created_utc'], unit='s')
    cmt_df['symbol']= symbol
    sort_by_date = cmt_df.sort_values(by=['created_utc'],ascending=False).reset_index(drop=True)
    
    
    return sort_by_date

StatementMeta(spark001, 0, 7, Finished, Available)

In [15]:
if __name__ == "__main__":
    # set up target ticker 
    symbol_list=['NVDA','AMD','INTC','QCOM','GOOG','MSFT','AMZN','AAPL']
    
    reddit = set_up_reddit()
    list_cm  = [extract_comment(search_post(symbol,reddit),symbol) for symbol in symbol_list]
    all_cm_df = pd.concat(list_cm)
    all_cm_df['collectedAt']= datetime.datetime.now().date()

    spark_df = spark.createDataFrame(all_cm_df)
    file_path ='abfss://files@datalake2gfrzms.dfs.core.windows.net/synapse/workspaces/synapse2gfrzms/warehouse/reddit_cms'

    spark_df.write.parquet(file_path,partitionBy='collectedAt',mode='overwrite')

StatementMeta(spark001, 0, 16, Finished, Available)